In [26]:
import ffisearch_B as ff
import numpy as np
import os
import common as cmn
from astropy.table import QTable, vstack
import lightkurve as lk
import re
import pandas as pd
from transitleastsquares import transit_mask

In [15]:
ticid = 1551634257
sigma_upper = 4.
sigma_lower = 12.
window_length = 0.8
method = 'biweight'
num_threads = 1

target_list = pd.read_csv("targets.csv")

tic_index = np.where(target_list['ID'] == ticid)[0][0]

mass = target_list['mass'][tic_index]
radius = target_list['rad'][tic_index]  

In [16]:
#using Tom's light curves
lc_direc = cmn.tom_lc_direc + str(ticid)[:-5].zfill(5) + 'XXXXX/lc-' + str(ticid).zfill(10) + "/"
dir_list = np.asarray(os.listdir(lc_direc))
light_curves2 = []
sectors = []
full_2d_list = []
for i in dir_list:
    data = pd.read_parquet(lc_direc+i)
    data_as = QTable.from_pandas(data)
    data_as['time'] = data_as['time'].jd - 2457000
    sector = re.findall('-s(.*)-', i)[0][:2]
    light_curves2.append(data_as)
    sectors.append(int(sector))
    full_2d_list.append([data_as, int(sector)])

In [17]:
light_curve_direc = cmn.light_curve_direc
save_direc = cmn.transit_search_direc

In [18]:
#using my light curves
#dir_list = np.asarray(os.listdir(light_curve_direc))
#get file names for the given TIC ID
#mask = np.asarray([(str(ticid).zfill(12) in i) for i in dir_list])
#masked_dir_list = dir_list[mask]
#Open the light curve files and 
#light_curves2 = []
#sectors = []
#full_2d_list = []
#for i in masked_dir_list:
    #Open the light curve using astropy
    #read_lc = QTable.read((light_curve_direc + i), format='fits', astropy_native=True)
    #Correct the time column to TESS times (BTJD)
    #read_lc['time'] = lk.LightCurve(read_lc).time.btjd
    #light_curves2.append(read_lc)
    #Determine the sector using the light curve file name
    #sector = re.findall('sector_00(.*)_tic', i)[0]
    #sectors.append(int(sector))
    #full_2d_list.append([read_lc, int(sector)])

In [19]:
flux_id = ff.determine_best_flux(light_curves2)

In [20]:
sorted_sectors = sorted(sectors)
consecutive_sectors = []
sec_group = [sorted_sectors[0]]
for i in range(1, len(sorted_sectors)):
    if sorted_sectors[i] == (sorted_sectors[i-1] + 1):
        sec_group.append(sorted_sectors[i])
    else:
        consecutive_sectors.append(sec_group)
        sec_group = [sorted_sectors[i]]
consecutive_sectors.append(sec_group)
light_curves = []
for i in consecutive_sectors:
    #stack the light curves using astropy vstack
    stack = vstack([full_2d_list[j][0] for j in range(len(full_2d_list)) if full_2d_list[j][1] in i])
    #Resort the new light curve so that it is in sequential order
    stack.sort('time')
    light_curves.append(stack)

In [21]:
all_flat = []
all_times = []
for i in range(len(light_curves)):
    time = light_curves[i]['time'].value
    flux = light_curves[i][flux_id].value
    times, flat, trend = ff.flatten_lightcurve(time, flux, sigma_upper, sigma_lower, window_length, method)
    all_flat.append(flat)
    all_times.append(times)

In [22]:
all_results = []
for i in range(len(light_curves)):
    results = ff.search_for_transit(all_times[i], all_flat[i], mass=mass, radius=radius, num_threads=num_threads)
    all_results.append(results)

In [ ]:
intransit = transit_mask(t, period, duration, T0)

In [23]:
if 0.2*len(all_results) > 2:
    sec_thresh1 = int(0.2*len(all_results))
else:
    sec_thresh1 = 2

flag, sec_num = ff.flagging_criteria(all_results, sde_thresh = 7, sec_thresh=sec_thresh1, save_direc=save_direc)

In [24]:
print(flag)

False


In [25]:
all_results[0]

{'SDE': 5.6430100475079525,
 'SDE_raw': 4.9772013739043235,
 'chi2_min': 1211.6229677882989,
 'chi2red_min': 0.9834602011268659,
 'period': 2.9201947262568106,
 'period_uncertainty': 0.008789703165573703,
 'T0': 1685.1571209770866,
 'duration': 0.10336239821273958,
 'depth': 0.998015595672346,
 'depth_mean': (0.9982816608711141, 0.00039159479385008256),
 'depth_mean_even': (0.9979364658343692, 0.0006170975879846659),
 'depth_mean_odd': (0.9986268559078593, 0.0004697451818232223),
 'transit_depths': array([0.99786296, 0.99760073, 0.99769909, 1.00040557,        nan,
        0.99853256, 0.99838465, 0.99796857, 0.99779917]),
 'transit_depths_uncertainties': array([0.00046921, 0.00100935, 0.00105731, 0.00076015,        nan,
        0.00042204, 0.0006592 , 0.00090777, 0.00206496]),
 'rp_rs': 0.04006104092494578,
 'snr': 4.884572836314652,
 'snr_per_transit': array([ 2.14775898,  2.41130335,  2.31245019, -0.40759971,  0.        ,
         1.4748019 ,  1.6234549 ,  2.04161829,  2.2118704 ]),
 